#### regression 1: treated x control (same cities)

In [10]:
import pandas as pd
import numpy as np
reg_data = pd.read_csv("data/cities_with_outcome.csv")

treated = reg_data[ (reg_data["year"] == 2018) & (reg_data["pct_new_users"] > 0.4)].index
reg_data = reg_data.loc[treated]

filter18 = reg_data[ (reg_data["year"] == 2018) & (reg_data["hiv_homo"] > 20)].index
reg_data = reg_data.loc[filter18]

reg_data["post"] = 0
reg_data.loc[reg_data["year"] == 2019, "post"] = 1

reg_data = (reg_data.reset_index().melt(
               id_vars=["name","code","year"],
               value_vars=["hiv_het_pct_change_follow_year",
                           "hiv_homo_pct_change_follow_year"],
               var_name="group")
)
reg_data.loc[reg_data["group"] == "hiv_het_pct_change_follow_year","treated"] = 0
reg_data.loc[reg_data["group"] == "hiv_homo_pct_change_follow_year","treated"] = 1

#reg_data.loc[reg_data["group"] == "hiv_het","treated"] = 0
#reg_data.loc[reg_data["group"] == "hiv_homo","treated"] = 1

reg_data = reg_data.query("year.isin([2015,2019])")

reg_data["pct_change"] = reg_data["value"]
#reg_data["log_cases"] = reg_data["value"].apply(np.log)

#import statsmodels.formula.api as smf
#
#smf.ols('log_cases ~ treated*post ', data=reg_data).fit().summary().tables[1]

In [8]:
reg_data

,name,code,year,group,value,treated,pct_change,log_cases


In [7]:
from linearmodels.panel import PanelOLS
mod = PanelOLS.from_formula("cases ~ treated*post",
                            data=reg_data.set_index(["code","year"]))

result = mod.fit()
# cov_type='clustered', cluster_entity=True
# statsmodels
result.summary.tables[1]

ValueError: Length of values (50607) does not match length of index (0)